In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag
from nltk.tree import Tree
import spacy
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from matplotlib import pyplot as plt
%matplotlib inline
from textblob import Word
from textblob import TextBlob
import gensim
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
globe_input_file = r"C:\Users\dukec\My-ML-Workspace\globe_6B\glove.6B.50d.txt"
globe_output_file = r"C:\Users\dukec\My-ML-Workspace\globe_6B\word2vec.txt"


#library that contains punctuation
import string
string.punctuation

#defining function for tokenization
import re

#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dukec\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_excel(r'C:\Users\dukec\My-ML-Workspace\Omdena Local Chapters\liberia-fake-news-detection\src\data\Full_Dataset.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33673 entries, 0 to 33672
Columns: 605 entries, title to Unnamed: 604
dtypes: datetime64[ns](1), float64(2), object(602)
memory usage: 155.4+ MB


In [5]:
df.head()

,title,content,date,Publisher,link,Label,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 595,Unnamed: 596,Unnamed: 597,Unnamed: 598,Unnamed: 599,Unnamed: 600,Unnamed: 601,Unnamed: 602,Unnamed: 603,Unnamed: 604
0,\nGovernment Must Provide Security for AML in ...,"Last week, armed men carrying machetes and oth...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/government-must-provide...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,\nLiberia Army Chief Of Staff Predicts Unfores...,"Amid recent coup in Niger, the Chief Staff of ...",2023-07-28 00:00:00,Gnn News Liberia,https://gnnliberia.com/liberia-army-chief-of-s...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,"\nGuns & Machetes: Men Attack, Vandalize Arcel...",A group of unknown men said to be from Zolowee...,2023-07-30 00:00:00,Gnn News Liberia,https://gnnliberia.com/guns-machetes-men-attac...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,\nPresident Weah commits to delivering violen...,"Liberian President, George Weah, has pledged t...",2023-07-27 00:00:00,Gnn News Liberia,https://gnnliberia.com/president-weah-commits-...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,\nNew World Bank Liberia Country Manager Arrives,"MONROVIA, July 31, 2023 â€“ The new World Bank...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/new-world-bank-liberia-...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [6]:
df.describe()

,Label,Unnamed: 363
count,33672.000000,1.0
mean,0.818395,2022.0
std,0.385524,NaN
min,0.000000,2022.0
25%,1.000000,2022.0
50%,1.000000,2022.0
75%,1.000000,2022.0
max,1.000000,2022.0


In [7]:
df.shape

(33673, 605)

In [8]:
df.isnull().sum()

title             565
content          1099
date              666
Publisher         822
link              428
                ...  
Unnamed: 600    33672
Unnamed: 601    33672
Unnamed: 602    33672
Unnamed: 603    33672
Unnamed: 604    33672
Length: 605, dtype: int64

In [9]:
columns = df.columns

In [10]:
for i in range(len(columns)):
    if(df[columns[i]].isnull().sum() == 33672):
        df.drop(columns[i], axis = 1, inplace = True)

In [11]:
df.shape

(33673, 289)

In [12]:
df.head()

,title,content,date,Publisher,link,Label,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 279,Unnamed: 280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287,Unnamed: 288
0,\nGovernment Must Provide Security for AML in ...,"Last week, armed men carrying machetes and oth...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/government-must-provide...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,\nLiberia Army Chief Of Staff Predicts Unfores...,"Amid recent coup in Niger, the Chief Staff of ...",2023-07-28 00:00:00,Gnn News Liberia,https://gnnliberia.com/liberia-army-chief-of-s...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"\nGuns & Machetes: Men Attack, Vandalize Arcel...",A group of unknown men said to be from Zolowee...,2023-07-30 00:00:00,Gnn News Liberia,https://gnnliberia.com/guns-machetes-men-attac...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,\nPresident Weah commits to delivering violen...,"Liberian President, George Weah, has pledged t...",2023-07-27 00:00:00,Gnn News Liberia,https://gnnliberia.com/president-weah-commits-...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\nNew World Bank Liberia Country Manager Arrives,"MONROVIA, July 31, 2023 â€“ The new World Bank...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/new-world-bank-liberia-...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
columns = df.columns

In [14]:
df[columns[4]].isnull().sum()

428

In [15]:
count  = 0
for i in range(len(columns)):
    if(df[columns[i]].isnull().sum() >= 0.1 * 33672):
        df.drop(columns[i], axis = 1, inplace = True)

In [16]:
df.shape

(33673, 6)

In [17]:
df.head(10)

,title,content,date,Publisher,link,Label
0,\nGovernment Must Provide Security for AML in ...,"Last week, armed men carrying machetes and oth...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/government-must-provide...,1.0
1,\nLiberia Army Chief Of Staff Predicts Unfores...,"Amid recent coup in Niger, the Chief Staff of ...",2023-07-28 00:00:00,Gnn News Liberia,https://gnnliberia.com/liberia-army-chief-of-s...,1.0
2,"\nGuns & Machetes: Men Attack, Vandalize Arcel...",A group of unknown men said to be from Zolowee...,2023-07-30 00:00:00,Gnn News Liberia,https://gnnliberia.com/guns-machetes-men-attac...,1.0
3,\nPresident Weah commits to delivering violen...,"Liberian President, George Weah, has pledged t...",2023-07-27 00:00:00,Gnn News Liberia,https://gnnliberia.com/president-weah-commits-...,1.0
4,\nNew World Bank Liberia Country Manager Arrives,"MONROVIA, July 31, 2023 â€“ The new World Bank...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/new-world-bank-liberia-...,1.0
5,\nWhat CPP Government Will Do Differently – Cu...,The Standard Bearer of the Collaborating Polit...,2023-08-01 00:00:00,Gnn News Liberia,https://gnnliberia.com/what-cpp-government-wil...,1.0
6,\nWeah Says Liberia’s Best Days Yet To Come As...,"Monrovia,Â Liberia â€“ Despite the horde of d...",2023-07-27 00:00:00,Gnn News Liberia,https://gnnliberia.com/weah-says-liberias-best...,1.0
7,\nPolice Arrests Woman Who Brutally Assaulted ...,The Liberia National Police says it has arrest...,2023-08-01 00:00:00,Gnn News Liberia,https://gnnliberia.com/police-arrests-woman-wh...,1.0
8,"\nNEC Declares Campaign Open On Saturday, As 2...",The National Elections Commission (NEC) has de...,2023-08-04 00:00:00,Gnn News Liberia,https://gnnliberia.com/nec-declares-campaign-o...,1.0
9,"\nCampaign Opens Today, Candidates Will Do All...",Over twenty presidential candidates including ...,2023-08-05 00:00:00,Gnn News Liberia,https://gnnliberia.com/campaign-opens-today-ca...,1.0


In [18]:
df.isnull().sum()

title         565
content      1099
date          666
Publisher     822
link          428
Label           1
dtype: int64

**Let's Check Fake and Real Labeled Data Point Count Before Cleaning**

In [19]:
fake_count = (df['Label'] == 0.0).sum()
real_count = (df['Label'] == 1.0).sum()

In [20]:
print("Fake Labelled Data Points: ", fake_count)

Fake Labelled Data Points:  6115


In [21]:
print("Real Labelled Data Points: ", real_count)

Real Labelled Data Points:  27557


In [22]:
# Without the content there is no need to keep those rows

df = df.dropna(subset=['content'])

In [23]:
df.isnull().sum()

title          2
content        0
date          57
Publisher    821
link           1
Label          1
dtype: int64

*After Removing all those NAN values now we can use our fake_count got decreased.*

- **So here we can go for two different analysis which will be without removing NAN values and with removing NAN values**
- **Or Maybe we have to go for Data Augmentation on the fake_data part**

In [24]:
fake_count = (df['Label'] == 0.0).sum()
real_count = (df['Label'] == 1.0).sum()

In [25]:
print("Fake Labelled Data Points: ", fake_count)

Fake Labelled Data Points:  5646


In [26]:
print("Real Labelled Data Points: ", real_count)

Real Labelled Data Points:  26927


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32574 entries, 0 to 33672
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      32572 non-null  object 
 1   content    32574 non-null  object 
 2   date       32517 non-null  object 
 3   Publisher  31753 non-null  object 
 4   link       32573 non-null  object 
 5   Label      32573 non-null  float64
dtypes: float64(1), object(5)
memory usage: 1.7+ MB


In [28]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['clean_content']= df['content'].apply(lambda x:remove_punctuation(str(x)))
df.head()

,title,content,date,Publisher,link,Label,clean_content
0,\nGovernment Must Provide Security for AML in ...,"Last week, armed men carrying machetes and oth...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/government-must-provide...,1.0,Last week armed men carrying machetes and othe...
1,\nLiberia Army Chief Of Staff Predicts Unfores...,"Amid recent coup in Niger, the Chief Staff of ...",2023-07-28 00:00:00,Gnn News Liberia,https://gnnliberia.com/liberia-army-chief-of-s...,1.0,Amid recent coup in Niger the Chief Staff of t...
2,"\nGuns & Machetes: Men Attack, Vandalize Arcel...",A group of unknown men said to be from Zolowee...,2023-07-30 00:00:00,Gnn News Liberia,https://gnnliberia.com/guns-machetes-men-attac...,1.0,A group of unknown men said to be from Zolowee...
3,\nPresident Weah commits to delivering violen...,"Liberian President, George Weah, has pledged t...",2023-07-27 00:00:00,Gnn News Liberia,https://gnnliberia.com/president-weah-commits-...,1.0,Liberian President George Weah has pledged to ...
4,\nNew World Bank Liberia Country Manager Arrives,"MONROVIA, July 31, 2023 â€“ The new World Bank...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/new-world-bank-liberia-...,1.0,MONROVIA July 31 2023 â€“ The new World Bank C...


In [29]:
df['content_lower'] = df['clean_content'].apply(lambda x: str(x).lower())

In [30]:
df.head()

,title,content,date,Publisher,link,Label,clean_content,content_lower
0,\nGovernment Must Provide Security for AML in ...,"Last week, armed men carrying machetes and oth...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/government-must-provide...,1.0,Last week armed men carrying machetes and othe...,last week armed men carrying machetes and othe...
1,\nLiberia Army Chief Of Staff Predicts Unfores...,"Amid recent coup in Niger, the Chief Staff of ...",2023-07-28 00:00:00,Gnn News Liberia,https://gnnliberia.com/liberia-army-chief-of-s...,1.0,Amid recent coup in Niger the Chief Staff of t...,amid recent coup in niger the chief staff of t...
2,"\nGuns & Machetes: Men Attack, Vandalize Arcel...",A group of unknown men said to be from Zolowee...,2023-07-30 00:00:00,Gnn News Liberia,https://gnnliberia.com/guns-machetes-men-attac...,1.0,A group of unknown men said to be from Zolowee...,a group of unknown men said to be from zolowee...
3,\nPresident Weah commits to delivering violen...,"Liberian President, George Weah, has pledged t...",2023-07-27 00:00:00,Gnn News Liberia,https://gnnliberia.com/president-weah-commits-...,1.0,Liberian President George Weah has pledged to ...,liberian president george weah has pledged to ...
4,\nNew World Bank Liberia Country Manager Arrives,"MONROVIA, July 31, 2023 â€“ The new World Bank...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/new-world-bank-liberia-...,1.0,MONROVIA July 31 2023 â€“ The new World Bank C...,monrovia july 31 2023 â€“ the new world bank c...


In [31]:
def tokenization(text):
    tokens = re.split('W+',text)
    return tokens
#applying function to the column
df['content_tokenied']= df['content_lower'].apply(lambda x: tokenization(str(x)))

In [32]:
df.head()

,title,content,date,Publisher,link,Label,clean_content,content_lower,content_tokenied
0,\nGovernment Must Provide Security for AML in ...,"Last week, armed men carrying machetes and oth...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/government-must-provide...,1.0,Last week armed men carrying machetes and othe...,last week armed men carrying machetes and othe...,[last week armed men carrying machetes and oth...
1,\nLiberia Army Chief Of Staff Predicts Unfores...,"Amid recent coup in Niger, the Chief Staff of ...",2023-07-28 00:00:00,Gnn News Liberia,https://gnnliberia.com/liberia-army-chief-of-s...,1.0,Amid recent coup in Niger the Chief Staff of t...,amid recent coup in niger the chief staff of t...,[amid recent coup in niger the chief staff of ...
2,"\nGuns & Machetes: Men Attack, Vandalize Arcel...",A group of unknown men said to be from Zolowee...,2023-07-30 00:00:00,Gnn News Liberia,https://gnnliberia.com/guns-machetes-men-attac...,1.0,A group of unknown men said to be from Zolowee...,a group of unknown men said to be from zolowee...,[a group of unknown men said to be from zolowe...
3,\nPresident Weah commits to delivering violen...,"Liberian President, George Weah, has pledged t...",2023-07-27 00:00:00,Gnn News Liberia,https://gnnliberia.com/president-weah-commits-...,1.0,Liberian President George Weah has pledged to ...,liberian president george weah has pledged to ...,[liberian president george weah has pledged to...
4,\nNew World Bank Liberia Country Manager Arrives,"MONROVIA, July 31, 2023 â€“ The new World Bank...",2023-07-31 00:00:00,Gnn News Liberia,https://gnnliberia.com/new-world-bank-liberia-...,1.0,MONROVIA July 31 2023 â€“ The new World Bank C...,monrovia july 31 2023 â€“ the new world bank c...,[monrovia july 31 2023 â€“ the new world bank ...


In [36]:
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [37]:
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [38]:
#applying the function
df['no_stopwords']= df['content_tokenied'].apply(lambda x:remove_stopwords(str(x)))

*Stop words that are present in the nltk library such as in, until, to, I, here are removed from the tokenized text and the rest are stored in the no_stopwords column.*

In [ ]:
#defining the object for stemming
porter_stemmer = PorterStemmer()
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
df['content_stemmed']=df['no_stopwords'].apply(lambda x: stemming(str(x)))

In [ ]:
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
df['content_lemmatized']=data['no_stopwords'].apply(lambda x:lemmatizer(str(x)))

In [ ]:
type(df['content'])

In [ ]:
stopwords = set(STOPWORDS)
wordcloud = WordCloud(
                        background_color='black',
                        stopwords=stopwords,
                        max_words=200, 
                        random_state=42).generate(str(df['content']))

plt.figure(figsize=(15,10))
plt.axis("off")
plt.title("Words frequented in text", fontsize=15)
plt.imshow(wordcloud.recolor(colormap= 'viridis' , random_state=42), alpha=0.98)
plt.show()